In [1]:
import numpy as np
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
import psutil
import time

In [2]:
matrix_path = 'dataset/stack-matrix.csv'
init_mask_path = 'dataset/init_stack_mask.npy'
zip_path = 'dataset/stack.zip'

In [3]:
matrix_df = pd.read_csv(matrix_path, index_col='filename')
matrix = matrix_df.to_numpy()
filenames_to_idx = {filename: idx for idx, filename in enumerate(matrix_df.index)}
print(filenames_to_idx)
matrix

{'00082a6a3ce8fc0108af4a699efe14e365143cc6': 0, '000e6d88a61ab15d3bd71dd9e538e5e5fbaaf754': 1, '00115a785f16c3cc2a3d8dd01cdaacbc1bad7978': 2, '003a5bea517a6cd64f28b76dbbf579c8399b8095': 3, '003ada1ac5a3c833ca69d4c20800127dc0e2edb8': 4, '004a3f9b78ec1f131dddbd5a05bd89ff42aafaba': 5, '006ad3e743b877a991463e0149f02e8c349a3c31': 6, '0082f3ed627d38d1dc2d01910f1f5b67cc12b51e': 7, '009700f05bebda7c4cd16e1abf48adfcbd602a92': 8, '009c962fb256adbe7340c34eb73c475763e98fad': 9, '00a9b48ae71d11f0eb14c3c9a960118d836314cc': 10, '00baea73feb83bdb63241e1d968e433931fe8f1f': 11, '00c312a55c7c44b331bd942fcf9f7b371d91bdb2': 12, '00c94dd90a4c2a4d798403119f2fe90effb73c47': 13, '00d3fc063df5cdc4f52e8fbef2e43f89d6a58a89': 14, '00d4e659d5ac45ef399ab0ece8b5ecb7a7c72ce0': 15, '00e7032267dbafaec14e68f25bf85a97bbc535e9': 16, '0105ad24f3195543a2ad30b67f909ab1f199d62e': 17, '012b8a493db00ee0c818d278f25660f7e9a06092': 18, '0159fb8ba294426b79d942acc03d17da9cc9935b': 19, '015a14ce18c3e040e1bc336a7df89f838119e18a': 20, '

array([[ 0.38916993,  0.4338932 ,  0.36465693, ...,  0.53655314,
         0.53655314, 10.38916993],
       [ 0.94215488,  0.9066596 ,  0.85972118, ...,  1.08565307,
         1.08565307, 10.94215488],
       [ 0.08604074,  0.08604074,  0.08804536, ...,  0.17228484,
         0.17228484,  0.22243524],
       ...,
       [ 0.06093884,  0.06323624,  0.05829144, ...,  0.06199193,
         0.06199193, 10.07263494],
       [ 0.06923032,  0.07244706,  0.06708717, ...,  0.07108331,
         0.07108331, 10.07769012],
       [ 0.04610276,  0.04721856,  0.04404068, ...,  0.04403234,
         0.04403234, 10.05048084]])

In [4]:
from zipfile import ZipFile
import json
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import torch
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch import optim
import random
import time
import os

# load all the plans from the zip file
i = 0
all_plans = []
with ZipFile(zip_path, "r") as f:
    plans = [x for x in f.namelist() if x.endswith(".json") and "MACOSX" not in x]
    for fp in tqdm(plans):
        with f.open(fp) as pfd:
            data = json.load(pfd)
            data["plan_tree"] = data["plan"][0][0][0]["Plan"]
            all_plans.append(data)

print("Loaded", len(all_plans), "query plans")

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])

        return self.parent[x]
    
    def union(self, x, y):
        xroot = self.find(x)
        yroot = self.find(y)

        if xroot == yroot:
            return
        
        if self.rank[xroot] < self.rank[yroot]:
            self.parent[xroot] = yroot
        elif self.rank[xroot] > self.rank[yroot]:
            self.parent[yroot] = xroot
        else:
            self.parent[yroot] = xroot
            self.rank[xroot] += 1
    
    def get_elements_in_set(self, x):
        xroot = self.find(x)
        return [i for i in range(len(self.parent)) if self.find(i) == xroot]


# determine mapping from plans to matrix positions
all_filenames = set(x["filename"] for x in all_plans)
filenames_to_idx = {fn: idx for idx, fn in enumerate(sorted(all_filenames))}

ufs = {}
for plan in all_plans:
    plan["matrix pos"] = {"row": filenames_to_idx[plan["filename"]],
                          "cols": plan["hint_list"]}
    i = plan["matrix pos"]["row"]
    hint_list = plan['hint_list']
    for j in range(len(hint_list)):
        col = hint_list[j]
        if i not in ufs:
            ufs[i] = UnionFind(49)
        for k in range(j+1, len(hint_list)):
            col2 = hint_list[k]
            ufs[i].union(col, col2)
            

100%|██████████| 95893/95893 [00:11<00:00, 8273.72it/s] 


Loaded 95893 query plans


In [5]:
init_mask = np.load(init_mask_path)
default_time = np.sum(matrix[:,0])

In [6]:
# add a new row to the matrix, the value is 18.004
new_row = np.array([576.502233] * matrix.shape[1])
matrix = np.vstack([matrix, new_row])
# new_row = np.array([81.251] * matrix.shape[1])
# matrix = np.vstack([matrix, new_row])
matrix

array([[3.89169931e-01, 4.33893204e-01, 3.64656925e-01, ...,
        5.36553144e-01, 5.36553144e-01, 1.03891699e+01],
       [9.42154884e-01, 9.06659603e-01, 8.59721184e-01, ...,
        1.08565307e+00, 1.08565307e+00, 1.09421549e+01],
       [8.60407352e-02, 8.60407352e-02, 8.80453587e-02, ...,
        1.72284842e-01, 1.72284842e-01, 2.22435236e-01],
       ...,
       [6.92303181e-02, 7.24470615e-02, 6.70871735e-02, ...,
        7.10833073e-02, 7.10833073e-02, 1.00776901e+01],
       [4.61027622e-02, 4.72185612e-02, 4.40406799e-02, ...,
        4.40323353e-02, 4.40323353e-02, 1.00504808e+01],
       [5.76502233e+02, 5.76502233e+02, 5.76502233e+02, ...,
        5.76502233e+02, 5.76502233e+02, 5.76502233e+02]])

In [7]:
new_mask_row = np.array([0] * matrix.shape[1])
new_mask_row[0] = 1
init_mask = np.vstack([init_mask, new_mask_row])
# new_mask_row = np.array([0] * matrix.shape[1])
# new_mask_row[0] = 1
# init_mask = np.vstack([init_mask, new_mask_row])
init_mask

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [8]:
ufs[matrix.shape[0]-1] = UnionFind(49)
print(matrix.shape[0]-1)

6191


In [9]:
def get_same_hints(q, hintset):
    return ufs[q].get_elements_in_set(hintset)

def get_exec_time(m, mask):
    observed = m * mask
    exec_time = 0
    groups = []
    n,d = m.shape
    for i in range(n):
        groups.append(set())
    for i in range(n):
        for j in range(d):
            if mask[i, j] == 1:
                group = ufs[i].find(j)
                if group not in groups[i]:
                    groups[i].add(group)
                    exec_time += observed[i, j]
    
    return exec_time

def get_p_49(mask):
    return mask.sum() / mask.size

def get_min_observed(m, mask):
    R = m * mask
    R[R == 0] = np.inf
    min_observed_latency = np.min(R, axis=1)
    return min_observed_latency

In [10]:
get_same_hints(6191, 0)

[0]

In [29]:
def greedy_timeout(output_path, new_observe_size = 32):
    mask = np.zeros_like(init_mask)
    for i in range(matrix.shape[0]):
        same_hints = get_same_hints(i, 0)
        mask[i, same_hints] = 1
    exec_time = get_exec_time(matrix, mask)
    min_observed = get_min_observed(matrix, mask)
    timeout_m = np.zeros(matrix.shape)
    timeout = 0
    results = []
    print("Execution time: ", exec_time)
    while True:
        exec_time = get_exec_time(matrix, mask) + timeout
        min_observed = get_min_observed(matrix, mask)
        
        results.append({"training_time": 0,
                        "inference_time": 0,
                        "exec_time": exec_time, 
                        "total_latency": np.sum(min_observed), 
                        "p50": np.median(min_observed), 
                        "p90": np.percentile(min_observed, 90), 
                        "p95": np.percentile(min_observed, 95), 
                        "p99": np.percentile(min_observed, 99)})
        
        with open(output_path, "w") as file:
            json.dump(results, file, indent=4)
        
        print("Total latency: ", np.sum(min_observed))
        print("Execution time: ", exec_time)
        
        cnt = 0
        selects = np.argsort(-min_observed)
        
        for i in range(len(selects)):
            if cnt >= new_observe_size:
                break
            file_i = selects[i]
            
            if mask[file_i].sum() == mask.shape[1]:
                continue
            
            while 1:
                hint_i = np.random.randint(matrix.shape[1])
                
                if mask[file_i, hint_i] == 0:
                    if timeout_m[file_i, hint_i] == 1:
                        continue
                    
                    same_hints = get_same_hints(file_i, hint_i)
                    
                    if matrix[file_i, hint_i] >= min_observed[file_i]:
                        timeout_m[file_i, same_hints] = 1
                        print("Timeout File {}, Hint {}, Real {}, Min observed {}".format(file_i, hint_i, matrix[file_i, hint_i], min_observed[file_i]))
                        timeout += min_observed[file_i]
                        break
                    
                    mask[file_i, same_hints] = 1
                    print("File {}, Hint {}, Real {}, Min observed {}".format(file_i, hint_i, matrix[file_i, hint_i], min_observed[file_i]))
                    cnt += 1
                break
            if exec_time > default_time + 3600*4:
                return 

In [30]:
for run in range(1,21):
    output_path = f'experiment/stack/greedy_test_{run}.json'
    open(output_path, 'w').close()
    greedy_timeout(output_path, new_observe_size = 1)

Execution time:  5848.939282388886
Total latency:  5848.939282388886
Execution time:  5848.939282388886
Timeout File 6191, Hint 24, Real 576.502233, Min observed 576.502233
File 5488, Hint 9, Real 20.06968116760254, Min observed 239.5308163166046
Total latency:  5629.478147239884
Execution time:  6445.511196556488
Timeout File 6191, Hint 37, Real 576.502233, Min observed 576.502233
File 5588, Hint 48, Real 1.52946138381958, Min observed 96.1565806865692
Total latency:  5534.851027937134
Execution time:  7023.542890940308
Timeout File 6191, Hint 45, Real 576.502233, Min observed 576.502233
Timeout File 1672, Hint 48, Real 77.6679584980011, Min observed 67.6679584980011
File 4265, Hint 10, Real 17.27454686164856, Min observed 32.28972339630127
Total latency:  5519.835851402481
Execution time:  7684.987629299958
Timeout File 6191, Hint 19, Real 576.502233, Min observed 576.502233
Timeout File 3098, Hint 46, Real 68.28120231628418, Min observed 58.28120231628418
Timeout File 4165, Hint 33,

In [24]:
def censored_als(X, mask, cutoffs, rank, niters, lambda_):
    """
    Alternating Least Squares algorithm for matrix factorization
    X: matrix to factorize
    mask: binary mask of observed entries
    rank: rank of the factorization
    niters: number of iterations
    lambda_: regularization parameter
    """
    n, m = X.shape
    A = np.random.rand(n, rank)
    B = np.random.rand(m, rank)
    for _ in range(niters):
        target = X + (1 - mask) * (np.dot(A, B.T))
        violations = (target < cutoffs) & (cutoffs > 0)
        target[violations] = cutoffs[violations]
        A = np.linalg.solve(np.dot(B.T, B) + lambda_ * np.eye(rank), np.dot(target, B).T).T
        A[A < 0] = 0
        target = X + (1 - mask) * (np.dot(A, B.T))
        violations = (target < cutoffs) & (cutoffs > 0)
        target[violations] = cutoffs[violations]
        B = np.linalg.solve(np.dot(A.T, A) + lambda_ * np.eye(rank), np.dot(target.T, A).T).T
        B[B < 0] = 0
    
    return X + (1 - mask) * (np.dot(A, B.T))

def censored_als_timeout(rank, lambda_, alpha, beta, output_path, new_observe_size=128):

        mask = init_mask.copy()
        exec_time = get_exec_time(matrix, mask)
        timeout_m = np.zeros_like(matrix)
        explored_m = init_mask.copy()
        min_observed = get_min_observed(matrix, mask)
        timeout = 0
        results = []
        
        while 1:
            exec_time = get_exec_time(matrix, mask) + timeout
            min_observed = get_min_observed(matrix, mask)
            
            masked_m = matrix * mask
            log_m = np.log1p(masked_m)
            log_timeout_m = np.log1p(timeout_m)
            
            start_time = time.time()
            pred_m = censored_als(log_m, mask, log_timeout_m, rank, 50, lambda_)
            training_time = time.time() - start_time
            pred_m = np.expm1(pred_m)
            mse = np.mean((pred_m - matrix) ** 2)
            pred_m = pred_m * (1-mask)
            pred_m[pred_m == 0] = np.inf
            start_time = time.time()
            mc_select = np.argmin(pred_m, axis=1)
            inference_time = time.time() - start_time
            
            results.append({"training_time": training_time,
                            "inference_time": inference_time,
                            "exec_time": exec_time, 
                            "total_latency": np.sum(min_observed), 
                            "p50": np.median(min_observed), 
                            "p90": np.percentile(min_observed, 90), 
                            "p95": np.percentile(min_observed, 95), 
                            "p99": np.percentile(min_observed, 99)})
            
            with open(output_path, "w") as file:
                json.dump(results, file, indent=4)
            
            # print("Total latency: ", np.sum(min_observed))
            # print("Execution time: ", exec_time)
            # print("MSE", mse)
            
            mc_min = np.min(pred_m, axis=1)
            improve = (min_observed - mc_min) / mc_min
            
            selects = np.argsort(-improve)
            cnt = 0
            
            print("6191:",mc_select[6191], mc_min[6191])
            for select in selects:
                if cnt >= new_observe_size:
                    break
                hint = mc_select[select]
                timeout_tolerance = min(alpha * min_observed[select], beta * pred_m[select, hint])
                # assert improve[select] == min_observed[select] - pred_m[select, hint]
                if np.isinf(pred_m[select, hint]) \
                    or explored_m[select, hint] != 0 \
                    or pred_m[select, hint] >= timeout_tolerance:
                    continue
                
                same_hints = get_same_hints(select, hint)
                
                if matrix[select, hint] >= min_observed[select]:
                    explored_m[select, same_hints] = 1
                
                if matrix[select, hint] >= timeout_tolerance:
                    timeout_m[select, same_hints] = timeout_tolerance
                    timeout += timeout_tolerance
                    print("Timeout File {}, Hint {}, Real {}, Min observed {}".format(select, hint, matrix[select, hint], timeout_tolerance))
                    continue
                
                mask[select,same_hints] = 1
                explored_m[select, same_hints] = 1
                cnt += 1
                print("File {}, Hint {}, Prediction {}, Min Observed {}, Actual {}".format(select, hint, pred_m[select, hint], min_observed[select], matrix[select, hint]))
            
            print("new observe size", new_observe_size - cnt)
            while cnt < new_observe_size:
                min_observed = get_min_observed(matrix, mask)
                file_i = np.random.randint(mask.shape[0])
                hint_i = np.random.randint(mask.shape[1])
                if mask[file_i, hint_i] == 0 \
                    and explored_m[file_i, hint_i] == 0:
                        
                    same_hints = get_same_hints(file_i, hint_i)
                    
                    if matrix[file_i, hint_i] >= min_observed[file_i]:
                        timeout += min_observed[file_i]
                        explored_m[file_i, same_hints] = 1
                        timeout_m[file_i, same_hints] =  min_observed[file_i]
                        continue
                    
                    explored_m[file_i, same_hints] = 1
                    mask[file_i, same_hints] = 1
                    cnt += 1
            if exec_time > default_time + 3600*4:
                return

In [33]:
rank = 15
lambda_ = 0.2
alpha = 1
beta = 10
new_observe_size = 16
for run in range(1,21):
    output_path = "experiment/stack/greedy_lime_{}.json".format(run)
    open(output_path, 'w').close()
    censored_als_timeout(rank, lambda_, alpha, beta, output_path, new_observe_size=new_observe_size)

/var/folders/sr/6vwss26n2kz07f4659hh1lp80000gq/T/ipykernel_50271/1667453620.py:48: RuntimeWarning: overflow encountered in expm1
  pred_m = np.expm1(pred_m)
/var/folders/sr/6vwss26n2kz07f4659hh1lp80000gq/T/ipykernel_50271/1667453620.py:49: RuntimeWarning: overflow encountered in square
  mse = np.mean((pred_m - matrix) ** 2)
/var/folders/sr/6vwss26n2kz07f4659hh1lp80000gq/T/ipykernel_50271/1667453620.py:73: RuntimeWarning: invalid value encountered in divide
  improve = (min_observed - mc_min) / mc_min


6191: 14 0.2420815170910071
Timeout File 6191, Hint 14, Real 576.502233, Min observed 2.4208151709100707
new observe size 16
6191: 40 10.857044370508671
Timeout File 6191, Hint 40, Real 576.502233, Min observed 108.57044370508672
Timeout File 1672, Hint 40, Real 77.6679584980011, Min observed 67.6679584980011
File 5488, Hint 9, Prediction 28.855119330789403, Min Observed 239.5308163166046, Actual 20.06968116760254
Timeout File 5588, Hint 9, Real 109.0856831073761, Min observed 96.1565806865692
Timeout File 3098, Hint 40, Real 68.28120231628418, Min observed 58.28120231628418
Timeout File 2558, Hint 9, Real 12.1131694316864, Min observed 11.092738151550291
File 4165, Hint 9, Prediction 10.812903884327492, Min Observed 30.7987220287323, Actual 14.088828563690186
Timeout File 4662, Hint 39, Real 22.600728034973145, Min observed 12.600728034973145
File 2329, Hint 9, Prediction 5.181172796252494, Min Observed 13.140854597091677, Actual 12.677170038223268
File 1951, Hint 9, Prediction 7.7202

/var/folders/sr/6vwss26n2kz07f4659hh1lp80000gq/T/ipykernel_50271/1667453620.py:83: RuntimeWarning: overflow encountered in scalar multiply
  timeout_tolerance = min(alpha * min_observed[select], beta * pred_m[select, hint])


6191: 40 278.5939889188016
new observe size 16
6191: 14 0.09500664499398347
new observe size 16
6191: 14 51.67472542836949
Timeout File 1951, Hint 39, Real 28.48784613609314, Min observed 12.107640743255615
Timeout File 2856, Hint 40, Real 21.176613569259644, Min observed 11.176613569259644
new observe size 16
6191: 14 2.060477194124532
File 3098, Hint 9, Prediction 12.911489897610638, Min Observed 20.160165071487427, Actual 17.75166130065918
File 1672, Hint 9, Prediction 14.143402820056348, Min Observed 20.433526277542114, Actual 17.85916757583618
new observe size 16
6191: 14 0.8908530872518751
Timeout File 4165, Hint 10, Real 15.65724277496338, Min observed 14.088828563690186
new observe size 16
6191: 14 8.82872935390307
Timeout File 5014, Hint 38, Real 20.878275394439697, Min observed 10.91488242149353
File 4265, Hint 9, Prediction 25.895638749178943, Min Observed 32.28972339630127, Actual 17.27454686164856
Timeout File 2856, Hint 38, Real 21.176613569259644, Min observed 11.1766135